Script to map the address list to the raw llc (col1 sender + col2 recipient)  
TomB 10/04/2023

In [1]:
import pandas as pd

In [2]:
dfn = pd.read_csv('../Data/HR_data_file.tsv', sep='\t')

In [3]:
#Select only in scope
dfn = dfn.query('SCOPE_INDICATOR == "Yes"')

In [4]:
#dfn

In [5]:
dfa = dfn.EMPL_EMAIL_ALIAS

In [6]:
#dfa

In [7]:
#Create the address file
dfa.to_csv('address.csv', index = False)

In [8]:
# split rows where needed
# each relationship on a new line
input_file_name = "../Data/pfizerv2_2.llc"
output_file_name = "temp_file.txt"

with open(input_file_name, "r") as input_file, open(output_file_name, "w") as output_file:
    for line in input_file:
        line_parts = line.strip().split("\t")
        sender = line_parts[0]
        recipients = line_parts[1].split(",")
        direction = line_parts[2]
        timestamp = line_parts[3]
        for recipient in recipients:
            new_line = f"{sender}\t{recipient}\t{direction}\t{timestamp}\n"
            output_file.write(new_line)

In [9]:
# with time taken
import re
import csv
import time

start_time = time.time()

#create an address file with all subjects in scope - one per line
# items from the address file must match at least once in the sender or reciever cols
with open("address.csv", "r") as csv_file:
    items_to_match = [row[0] for row in csv.reader(csv_file)]


# Open the file to read from - this is the llc c file from the client
with open("temp_file.txt", "r") as input_file, open("output_file.txt", "w") as output_file:

    # Define a regular expression to match any of the items to match
    regex = re.compile("|".join(items_to_match))

    # Process the input file in chunks
    while True:
        chunk = input_file.readlines(10000)  # Read 10,000 lines at a time
        if not chunk:
            break  # End of file

        # Filter the chunk to find lines that contain at least one item to match
        filtered_chunk = [line for line in chunk if regex.search(line)]

        # Write the filtered lines to the output file
        output_file.writelines(filtered_chunk)

end_time = time.time()

# Print the time taken in hours and minutes
elapsed_time = end_time - start_time
hours = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)
print(f"Time taken: {hours} hours {minutes} minutes")


KeyboardInterrupt: 